In [ ]:
def decorator(func):
    return func

@decorator # синтаксис декоратора
def decorated():
    print('Hello!')


In [ ]:
def decorator(func):
    def new_func():
        print('123')
    return new_func

@decorator
def decorated():
    print('Hello!')

decorated()
print(decorated.__name__)

123
new_func


Пример: написать декоратор, который записывает в лог результат декорируемой функции. В этом примере с помощью декоратора logger мы подменяем декорируемую функцию функцией wrapped. Эта функция принимает на вход тот же num_list и возвращает
тот же результат, что и исходная функция, но кроме этого записывает результат в логфайл.

In [ ]:
def logger(func):
    def wrapped(num_list):
        result = func(num_list)
        with open('log.txt', 'w') as f:
            f.write(str(result))

        return result
    return wrapped

@logger
def summator(num_list):
    return sum(num_list)
    
print('Summator: {}'.format(summator([1, 2, 3, 4])))

Summator: 10


In [ ]:
def logger(func):
    def wrapped(*args, **kwargs):
        result = func(*args, **kwargs)
        with open('log.txt', 'w') as f:
            f.write(str(result))
        return result
    return wrapped


Из-за того, что с помощью декоратора мы подменили функцию, её имя поменялось.

In [ ]:
print(summator.__name__)

wrapped


Этот факт иногда мешает при отладке. Чтобы такого не происходило, можно использовать декоратор wraps из модуля functools. Он подменяет определённые аргументы, docstring-и и названия так, что функция не меняется:


In [ ]:
import functools
def logger(func):
    @functools.wraps(func)
    def wrapped(*args, **kwargs):
        result = func(*args, **kwargs)
        with open('log.txt', 'w') as f:
            f.write(str(result))
        return result
    return wrapped

@logger
def summator(num_list):
    return sum(num_list)

print(summator.__name__)

summator


In [ ]:
def fff(func):
    print('Инициализация декоратора')
    def jjj(vvv):
        print(vvv)
    return jjj

@fff
def ggg(vvv):
    print('333')
    return 333

ggg(888)
ggg(999)
ggg(101010)

Инициализация декоратора
888
999
101010


Более сложная задача: написать декоратор с параметром, который записывает лог в
указанный файл. Для этого logger должен принимать имя файла и возвращать декоратор, который принимает функцию и подменяет её функцией wrapped, как мы делали до
этого. Всё просто:

In [ ]:
def logger(filename):
    def decorator(func):
        def wrapped(*args, **kwargs):
            result = func(*args, **kwargs)
            with open(filename, 'w') as f:
                 f.write(str(result))
            return result
        return wrapped
    return decorator

@logger('new_log.txt')
def summator(num_list):
    return sum(num_list)
# без синтаксического сахара:
# summator = logger('log.txt')(summator)
summator([1, 2, 3, 4, 5, 6])

with open('new_log.txt', 'r') as f:
    print(f.read())

21


Посмотрим, что будет, если применить сразу несколько декораторов:

In [ ]:
def first_decorator(func):
    def wrapped():
        print('Inside first_decorator product')
        return func()
    return wrapped

def second_decorator(func):
    def wrapped():
        print('Inside second_decorator product')
        return func()
    return wrapped

In [ ]:
@first_decorator
@second_decorator
def decorated():
    print('Finally called...')
# то же самое, но без синтаксического сахара:
# decorated = first_decorator(second_decorator(decorated))
decorated()

Inside first_decorator product
Inside second_decorator product
Finally called...


Видим, что сначала вызвался сначала первый декоратор, потом второй. Разберём это
подробнее. Функция second_decorator возвращает новую функцию wrapped, таким образом, функция подменяется на wrapped внутри second_decorator-а. После этого вызывается first_decorator, который принимает функцию полученную из
second_decorator-а wrapped и возвращает ещё одну функцию wrapped заменяя decorated
на неё. Таким образом, итоговая функция decorated — это функция wrapped из first_decorator-а,
вызывающая функцию из second_decorator-а.

Ещё один пример на применение декораторов. Обратите внимание, что сначала теги
идут в том же порядке, что и декораторы, а затем в обратном. Это происходит потому, что
декораторы вызываются один внутри другого.

In [ ]:
def bold(func):
    def wrapped():
        return "<b>" + func() + "</b>"
    return wrapped
    
def italic(func):
    def wrapped():
        return "<i>" + func() + "</i>"
    return wrapped

@bold
@italic
def hello():
    return "hello world"
# hello = bold(italic(hello))
print(hello())

<b><i>hello world</i></b>


In [1]:
def dec(func):
    print('dec')
    def decorator():
        print("1")
        func()
        print("2")
    return decorator

@dec
def deco():
    print('ffffffff')

dec


In [2]:
deco()

1
ffffffff
2


In [3]:
def dec():
    print('dec1')
    def decorator(func):
        print('dec2')
        def dec2():
            print("1")
            func()
            print("2")
        return dec2
    return decorator

@dec()
def deco():
    print('ffffffff')

dec1
dec2


In [4]:
deco()

1
ffffffff
2


In [5]:
def f1(a):
    print('1', a)
    return a+1

def f2(a):
    print('2', a)
    return a+1

def f3(a):
    print('3', a)
    return a

f = f3(f2(f1(5)))

1 5
2 6
3 7


In [6]:
class disp():
    def dec():
        print('dec')
        def decorator(func):
            print('decorator', func.__name__)
            def dec2():
                print("1")
                func()
                print("2")
            return dec2
        return decorator

@disp.dec()
def echo():
    print('3')

dec
decorator echo


In [7]:
echo()

1
3
2
